# Tiny few shot classifer

Im folgenden wird ein few shot classifier für multilabels im [SetFit-Frame](https://huggingface.co/blog/setfit) Format erzeugt. Gearbeitet wird mit `Python 3.12.3`.

# Pakete laden

In einem ersten Schritt werden die nötigen Pakete geladen. 

In [9]:
from datasets import load_dataset  # Zum Laden von Datensätzen aus der Hugging Face `datasets` Bibliothek
from sentence_transformers.losses import CosineSimilarityLoss  # Verlustfunktion basierend auf der Kosinusähnlichkeit, für Aufgaben wie Textähnlichkeit

from setfit import SetFitModel, SetFitTrainer  # Zum Laden des SetFit-Modells und Trainers, speziell für Few-Shot Learning mit Satztransformern
import pandas as pd  # Pandas für Datenmanipulation und -analyse
from sklearn.preprocessing import LabelEncoder  # Zum Encoden von Labels in numerische Werte für maschinelles Lernen
import os  # Für den Umgang mit Betriebssystemfunktionen (z.B. Dateipfade)
from datasets import Dataset, DatasetDict  # Zum Erstellen und Verwalten von Datensätzen in Hugging Face `datasets`-Format
from sklearn.model_selection import train_test_split  # Zum Aufteilen der Daten in Trainings- und Testsets

from transformers import TrainingArguments  # Zum Festlegen von Trainingsparametern für Modelle in der Transformers-Bibliothek

import torch  # Ermöglicht die GPU-Nutzung und effiziente Verarbeitung von Tensoren
import numpy as np  # Für numerische Berechnungen und Arbeiten mit Arrays

import re
import html

from unidecode import unidecode

# Arbeitsverzeichnis

Es wird weiterhin das Rootverzeichnis bestimmt.

In [10]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification


# Daten

## Daten laden

Das zur Zeit keine Daten für das trainieren eines multilable Classifiers vorliegen, werden die Trainings-Daten genutzt, die ursprünglich von Franziska Weber und Felix Süßenbach erzeugt wurden.

In [11]:
# Laden der Excel-Datei
df = pd.read_excel('data/train_data_franziska.xlsx')
 
# 1. Ersetze NaN-Werte durch 0
df = df.fillna(0)

# 2. Ersetze unendliche Werte durch 0
df.replace([np.inf, -np.inf], 0, inplace=True)

df[df.select_dtypes(include='number').columns] = df.select_dtypes(include='number').astype('int64')

 
df_cleaned = df[~(df.select_dtypes(include='number').eq(0).all(axis=1))]



# Für die weitere Verarbeitung werden die Daten ins  Hugging Face Dataset-Format transformiert. 

# /////////////////////////////////////////////////
# Hinweis: Das Dataset-Format der Hugging Face datasets-Bibliothek basiert auf Apache Arrow und ist speichereffizient, was schnelles Laden und Verarbeiten großer Datensätze ermöglicht. Es integriert sich nahtlos in Machine-Learning-Workflows und unterstützt einfache Transformationen und Vorverarbeitung durch Methoden wie .map(). Zudem ist es flexibel und kompatibel mit Pandas DataFrames sowie gängigen Frameworks wie PyTorch und TensorFlow.
#//////////////////////////////////////////////////

dataset = Dataset.from_pandas(df)
print(dataset)
 

Dataset({
    features: ['sentence', 'Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten', 'Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 'Dialog- und Konfliktfähigkeit'],
    num_rows: 1577
})


## Daten verarbeiten

### Feature Auswahl

In einem ersten Schritt werden die Features des Classifiers ausgewählt. Da sich insbesondere *klassische* und *transformative Future Skills* als schwer nachvollziehbar in den Daten erwiesen haben, werden diese aus dem Classifier ausgeschlossen, ebenso wie die Kurstitel und -beschreibungen (`sentence`). 

In [12]:
# ziehe column names als Liste
features = dataset.column_names

# definiere skills, die nicht vom classifier berücksichtigt werden
to_remove = ['sentence','Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 
             'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 
             'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 
             'Dialog- und Konfliktfähigkeit']

# schließe die oben definierten Variablen aus der Feature-Liste aus
filtered_features = [feature for feature in features if feature not in to_remove]

features = filtered_features

print(features)

['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten']


### One-Hot-Encoding

Für das Trainieren des Classifiers wird im folgenden ein *One-Hot-Encoding*-Array der Features erzeugt. Dies ist deshalb nötig, da im folgenden ein Modell für ein Multi-Label-Szenario trainiert werden soll: Ein Kursangebot kann den erwerb mehrerer Future Skills versprechen, etwa Softwareentwicklung **&** IT-Architektur. Das One-Hot-Encoding wird benötigt, um die Labels der Features in eine Form zu bringen, die das Classifier Modell von SetFit korrekt verarbeiten kann. Es stellt sicher, dass alle Klassen gleichwertig behandelt werden, und ermöglicht die Berechnung von Ähnlichkeiten verschiedener Kursinhalte.

Um den *One-Hot-Encoding*-Array zu erzeugen definieren wir `encode_labels(record)`: Diese Funktion geht durch jede Zeile eines Data Frames und erstellt eine neue Spalte namens `labels`. In dieser Spalte werden die Werte aus den oben definierten "features" gesammelt und in eine Liste gepackt. Diese Liste enthält Werte der Features.

Beispiel: Mit Blick auf die Features 'Data Analytics & KI', 'Softwareentwicklung' und 'Nutzerzentriertes Design', könnte ein Zeilenvektor des Arrays eines Kurses der sich mit Data Analytics & KI / Softwareentwicklung beschäftigt so aussehen: `[1, 1, 0]`

In [13]:
# generiere Funktion für One-Hot-Encoding für jede Zeile (record)
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}

# wende Funktion auf train_data an
dataset = dataset.map(encode_labels)

# um die Daten übersichtlich im viewer zu betrachten: umwandlung und pandas data frame
pd_dataset = dataset.to_pandas()

c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/1577 [00:00<?, ? examples/s]

### Erstellung von Test- und Trainingsdaten

Im folgenden wird in `dataset` für jedes Feature (`f`) in der Liste `features` nach den Zeilen gesucht, in denen das Feature einen gültigen (nicht-leeren oder nicht-null) Wert hat. Aus diesen Zeilen (Indizes) werden dann zufällig 8 Beispiele ausgewählt. `np.where(dataset[f])` gibt die Indizes der Zeilen zurück, bei denen der Wert in der Spalte `f` `true` ist (also weder leer noch null). Wenn der Wert `false` ist (z.B. `0`, `NaN`, `None`, `False`, ein leerer String ''), wird diese Zeile ausgeschlossen.

In [14]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset[f])[0], num_samples) for f in features]
)
print(samples)

[ 801  838  862 1398  838 1245 1429 1424 1450  625 1136 1107  829 1109
 1154  739 1518  964 1426 1576 1572 1574 1576 1481  809  529  325   96
 1178  755  690  974  992 1471  320  804  112 1483  992  382 1470 1470
   93 1466 1376 1470  571 1466  835 1495 1356 1452 1504 1495 1495  825
  248 1485 1485 1402 1508  248 1432 1512 1189  928 1189 1189  836 1189
  836  812  398  810  825  398 1003  952  825  810 1553 1549 1548 1546
 1541 1550 1548 1555]


Im Folgenden wird einerseits der Trainingsdatensatz `train_dataset` anhand der oben definierten Indizes aus dem `samples`-Objekt erzeugt. Andererseits wird ein komplementärer Datensatz `eval_dataset` erzeugt, mit allen Indizes die nicht im `samples`-Objekt vorkommen. 

In [15]:
train_dataset = dataset.select(samples)
eval_dataset = dataset.select(
    np.setdiff1d(np.arange(len(dataset)), samples)
)

 
# check data with pandas
pd_train_dataset = train_dataset.to_pandas()
pd_eval_dataset = eval_dataset.to_pandas()


## Modelltraining

Es wird das vortrainierte sentence-transformers-Modell `paraphrase-mpnet-base-v2` aus der Hugging Face Bibliothek geladen. Durch die Angabe der Strategie `one-vs-rest` wird das Modell für eine Multi-Label-Klassifikation konfiguriert, bei der jede Klasse eines Kurses einzeln vorhergesagt wird. Auf diese Weise wird es möglich, mehrere Skills pro Kurs zu identifizieren. 

In [16]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
 
model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Der `SetFitTrainer` wird instanziiert, um das Modell auf den Trainings- und Evaluierungsdatensätzen zu trainieren. Eine Bewertung des Modells wird erst zu einem späteren Zeitpunkt möglich, wenn Testdaten vorliegen.

1. Das vortrainierte `model` (siehe oben) wird als Grundlage für das Training verwendet, das auf dem Few-Shot Learning-Ansatz basiert. `paraphrase-mpnet-base-v2` ist ein vortrainiertes Modell aus der Sentence Transformers-Familie, das speziell für die Aufgabe der semantischen Textähnlichkeit und Paraphrasenerkennung entwickelt wurde. 
2. Der `train_dataset` enthält die Trainingsdaten, und der `eval_dataset` wird zu einem späteren Zeitpunkt genutzt, um die Klassifikations-Leistung des Modells während des Trainings zu bewerten.
3. Die Verlustfunktion (`loss_class`) wird als *Cosine Similarity Loss* festgelegt. Die *Cosine Similarity Loss* ist eine Verlustfunktion, die darauf basiert, wie ähnlich sich zwei Vektoren sind. In diesem Fall werden die Ausgabevektoren des Modells und die Zielvektoren (also die "Labels") mithilfe der Kosinus-Ähnlichkeit verglichen. Je höher die Ähnlichkeit also, desto besser das Modell. 
4. Die `column_mapping` gibt an, welche Spalten in den Datensätzen die Texte (`sentence` → `text`) und die zugehörigen Labels (`labels` → `label`) enthalten, sodass das Modell die Daten korrekt interpretieren kann.


In [17]:

trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset, 
    #eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "labels": "label"},
)

C:\Users\Hueck\AppData\Local\Temp\ipykernel_30092\2973711996.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset


Map:   0%|          | 0/1577 [00:00<?, ? examples/s]

Nun kann das Training beginnen: 

In [18]:
trainer.train()

***** Running training *****
  Num unique pairs = 63080
  Batch size = 16
  Num epochs = 1


  0%|          | 0/3943 [00:00<?, ?it/s]

{'embedding_loss': 0.2958, 'grad_norm': 2.32528018951416, 'learning_rate': 5.063291139240507e-08, 'epoch': 0.0}
{'embedding_loss': 0.2471, 'grad_norm': 1.836930751800537, 'learning_rate': 2.5316455696202535e-06, 'epoch': 0.01}
{'embedding_loss': 0.1602, 'grad_norm': 1.1381397247314453, 'learning_rate': 5.063291139240507e-06, 'epoch': 0.03}
{'embedding_loss': 0.0884, 'grad_norm': 1.2985222339630127, 'learning_rate': 7.5949367088607605e-06, 'epoch': 0.04}
{'embedding_loss': 0.056, 'grad_norm': 1.9718166589736938, 'learning_rate': 1.0126582278481014e-05, 'epoch': 0.05}
{'embedding_loss': 0.0465, 'grad_norm': 1.1419466733932495, 'learning_rate': 1.2658227848101268e-05, 'epoch': 0.06}
{'embedding_loss': 0.0431, 'grad_norm': 0.7483248114585876, 'learning_rate': 1.5189873417721521e-05, 'epoch': 0.08}
{'embedding_loss': 0.0285, 'grad_norm': 0.5731155872344971, 'learning_rate': 1.7721518987341772e-05, 'epoch': 0.09}
{'embedding_loss': 0.0224, 'grad_norm': 0.3748095631599426, 'learning_rate': 1.

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'embedding_loss': 0.0271, 'grad_norm': 0.2962232828140259, 'learning_rate': 1.9126268320180385e-05, 'epoch': 0.14}
{'embedding_loss': 0.0223, 'grad_norm': 0.4222840666770935, 'learning_rate': 1.8844419391206314e-05, 'epoch': 0.15}
{'embedding_loss': 0.0314, 'grad_norm': 0.4079793691635132, 'learning_rate': 1.8562570462232244e-05, 'epoch': 0.16}
{'embedding_loss': 0.0213, 'grad_norm': 0.45299822092056274, 'learning_rate': 1.8280721533258174e-05, 'epoch': 0.18}
{'embedding_loss': 0.0176, 'grad_norm': 1.076499104499817, 'learning_rate': 1.7998872604284104e-05, 'epoch': 0.19}
{'embedding_loss': 0.0235, 'grad_norm': 0.24857784807682037, 'learning_rate': 1.7717023675310034e-05, 'epoch': 0.2}
{'embedding_loss': 0.0227, 'grad_norm': 0.24881407618522644, 'learning_rate': 1.7435174746335967e-05, 'epoch': 0.22}
{'embedding_loss': 0.0143, 'grad_norm': 0.28781282901763916, 'learning_rate': 1.7153325817361894e-05, 'epoch': 0.23}
{'embedding_loss': 0.0268, 'grad_norm': 0.4491312801837921, 'learning_

...

{'embedding_loss': 0.0132, 'grad_norm': 0.14148910343647003, 'learning_rate': 5.033821871476888e-06, 'epoch': 0.77}
{'embedding_loss': 0.0134, 'grad_norm': 0.20920413732528687, 'learning_rate': 4.751972942502819e-06, 'epoch': 0.79}
{'embedding_loss': 0.0109, 'grad_norm': 0.15351445972919464, 'learning_rate': 4.470124013528749e-06, 'epoch': 0.8}
{'embedding_loss': 0.013, 'grad_norm': 0.16835469007492065, 'learning_rate': 4.188275084554679e-06, 'epoch': 0.81}
{'embedding_loss': 0.015, 'grad_norm': 1.0735487937927246, 'learning_rate': 3.9064261555806094e-06, 'epoch': 0.82}
{'embedding_loss': 0.0071, 'grad_norm': 0.2073952555656433, 'learning_rate': 3.6245772266065393e-06, 'epoch': 0.84}
{'embedding_loss': 0.0118, 'grad_norm': 0.1857868880033493, 'learning_rate': 3.342728297632469e-06, 'epoch': 0.85}
{'embedding_loss': 0.0107, 'grad_norm': 0.07779091596603394, 'learning_rate': 3.0608793686583995e-06, 'epoch': 0.86}
{'embedding_loss': 0.0106, 'grad_norm': 0.11103897541761398, 'learning_rate

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'embedding_loss': 0.0132, 'grad_norm': 1.1034280061721802, 'learning_rate': 2.2153325817361895e-06, 'epoch': 0.9}
{'embedding_loss': 0.0086, 'grad_norm': 0.32186993956565857, 'learning_rate': 1.9334836527621198e-06, 'epoch': 0.91}
{'embedding_loss': 0.0114, 'grad_norm': 0.0630861446261406, 'learning_rate': 1.6516347237880499e-06, 'epoch': 0.93}
{'embedding_loss': 0.0074, 'grad_norm': 0.2523956596851349, 'learning_rate': 1.3697857948139797e-06, 'epoch': 0.94}
{'embedding_loss': 0.0067, 'grad_norm': 0.23587703704833984, 'learning_rate': 1.0879368658399098e-06, 'epoch': 0.95}
{'embedding_loss': 0.0093, 'grad_norm': 0.42972636222839355, 'learning_rate': 8.0608793686584e-07, 'epoch': 0.96}
{'embedding_loss': 0.0086, 'grad_norm': 0.17302197217941284, 'learning_rate': 5.242390078917701e-07, 'epoch': 0.98}
{'embedding_loss': 0.0106, 'grad_norm': 0.1944885104894638, 'learning_rate': 2.423900789177001e-07, 'epoch': 0.99}


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'train_runtime': 1361.328, 'train_samples_per_second': 46.337, 'train_steps_per_second': 2.896, 'train_loss': 0.02162495925134431, 'epoch': 1.0}


In [ ]:
# # Modell evaluieren (Nur bei vorhandensein von Testdaten)
# metrics = trainer.evaluate()
# metrics


In [ ]:
trainer.push_to_hub("Chernoffface/fs-setfit-multilable-model")

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")

In [ ]:
preds = model(
    [
        "Die Einzelthemen umfassen: * Hard- and Software-Architecture of Modern Game Systems * Time Management in Milliseconds * Asset Loading and Compression * Physically Based Realtime Rendering and Animations * Handling of Large Game Scenes * Audio Simulation and Mixing * Constraint-Based Physics Simulation * Artificial Intelligence for Games * Multiplayer-Networking * Procedural Content Creation * Integration of Scripting Languages * Optimization and parallelization of CPU and GPU Code Die Übungen enthalten Theorie- und Praxisanteile.",
        "Aim/ learning outcomes:• learning of the programming language C and understanding of basic concepts of programming• finding and correcting programming errors• development of computer programs and organization of complex projects• working with software libraries• independent analysis of scientific problems and their implementation in CContent:Linux basics, the C++ programming language (e.g. data types, loops, functions, classes, templates), compiler (function, process), OpenSource tools (e.g. make, gnuplot), implementation of numerical algorithms as application examples",
        "scrum",
        "Automation Projects, Automation Methods, Development Methods for Automation Systems, Automation with Qualitative Models, Safety and Reliability of Automation Systems"
    ]
)
[[f for f, p in zip(features, ps) if p] for ps in preds]

 